# q3dfit notebook: rest-frame MIR, JWST-MIRI/MRS data of PDS 456, all channels, from [Seebeck et al. 2024](https://ui.adsabs.harvard.edu/abs/2024arXiv240918086S/abstract)

<h3><font color='teal'>Installation of the environment and the package are described <a href="https://q3dfit.readthedocs.io/">here</a>. </font></h3>

This Jupyter notebook allows you to run Q3Dfit, a PSF decomposition and spectral analysis package tailored for JWST NIRSpec and MIRI IFU observations. 

Q3Dfit is developed as a science-enabling data product by the Early Release Science Team #1335 Q3D. You can find more information about this ERS program **Q3D** [here](https://wwwstaff.ari.uni-heidelberg.de/dwylezalek/q3d.html) and [here](https://www.stsci.edu/jwst/science-execution/approved-programs/dd-ers/program-1335).

The software is based on the existing package IFSFIT developed by Dave Rupke (see [ADS link](https://ui.adsabs.harvard.edu/abs/2017ApJ...850...40R/abstract)).

The following notebook will guide you through the initialization procedure and will then perform the analysis. 

## Table of Contents

* [1. Initialization](#chapter1)
    * [1.0. Setting up the directory tree](#chapter1_0)
    * [1.1. Initializing the fit](#chapter1_1)
    * [1.2. Setting up the data and models](#chapter1_2)
    * [1.3. Setting up the fitting parameters](#chapter1_3)
        * [1.3.1. Emission line parameters](#chapter1_3_1)
        * [1.3.2. Continuum parameters](#chapter1_3_2)
* [2. Run fit](#chapter2)
* [3. Plot fit results](#chapter3)

## 1. Initialization <a class="anchor" id="chapter1"></a>

In [ ]:
import os.path
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
# Be sure to set the path to q3dfit correctly.
# For instance:
#import sys
#sys.path.append('/Users/jwstuser/q3dfit/')
#import sys
#sys.path.append("../")

### 1.0. Setting up the directory tree <a class="anchor" id="chapter1_0"></a>

Define the directories in which the data cube(s) that you want to analyse are stored and the output directories. We recommend creating a working directory that you name after your target, in which all outputs from q3dfit will be saved. Then download test data.

In [ ]:
# Base directory (book-keeping)
volume = 'pds456/'
# prefix label for output files
label = 'pds456'
# Input directory
indir = volume
if not os.path.exists(indir):
    os.makedirs(indir)
# Output directory
outdir = volume
if not os.path.exists(outdir):
    os.makedirs(outdir)
# Initialization file (q3di.npy) directory
initdir = volume
# Output logfile
logfile = outdir+label+'-fitlog.txt'

Download data from public Box folder. <font color='red'> Note: This also downloads the *.cf configuration file (specifying which spectral components to include in the MIR fitting). We note that the format of this file is currently under development and it will in the future be superseded by a more readable/convenient option. For a current description of continuum fitting parameters in the .cf file, please see the documentation: 
https://q3dfit.readthedocs.io/en/latest/

In [ ]:
# make tuples of urls and download filenames
# infile = 1x1 mock data cube: containing only 1 single spaxel with a Spitzer spectrum
# cf = config file
infile_tup = ('https://rhodes.box.com/shared/static/gapv1231zoe2eo4ndhm23b6lzjf5lm7j.fits', 'pds456.fits')
cf_tup = ('https://rhodes.box.com/shared/static/eys0cwiqc6sbc46fjwxp44iwkzzd7fez.cf', 'pds456.cf')
# download files; by default don't force overwrite and take first element of output
from q3dfit.jnb import download_files
infile = download_files(infile_tup, indir, force=False)[0]
cfinfile = download_files(cf_tup, indir, force=False)[0]
# add subdirectory to filenames
infile = indir+infile
cfinfile = indir+cfinfile

### 1.1. Initializing the fit <a class="anchor" id="chapter1_1"></a>

The initial parameters of the fit are stored in an object of class `q3din`. Each parameter or attribute of this class controls some aspect of the fit process. We start by instantiating the class. The only required parameters at the outset are the input data cube and label; the label is used for output file naming. 

The default JWST pipeline output has data, variance, and data quality in extensions 1, 2, and 3, respectively. Our processed cube has a different set of extensions, so we specify them here.

In [ ]:
from q3dfit.q3din import q3din
q3di = q3din(infile, label, dqext=None, outdir=outdir, logfile=logfile)

Here's a list of the fit parameters that are automatically set:

In [ ]:
q3di.__dict__

### 1.2. Setting up the data and models <a class="anchor" id="chapter1_2"></a>

Some general information about your cube. `argsreadcube` is a dictionary of attributes sent to the `Cube` class.
- For non-JWST data, set `wmapext` to `None`. The WMAP extension is a [3-D weight image](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/science_products.html) giving the relative weights of the output spaxels. While our data is from JWST, the wmap extension has been cut out during processing.
- Microns are the wavelength unit used internally, but `q3dfit` can accept input/output in Å.
- `q3dit` does calculations in f$_\lambda$ space, but assumes input units of MJy/sr, the JWST default. Other input wavelength units can be specified. In this case, the reduced data has been converted to erg/s/cm$^2$/Å. The output flux units will be in erg/s/cm$^2$/$\mu$m.



Some general information about your cube:

In [ ]:
from q3dfit.readcube import Cube
norm = 1e-13 # needs to be adjusted so the scale of the plot in the cell below is ~1
q3di.argsreadcube = {'wmapext': None,
                     'zerodq': True,
                     'usebunit':True,
                     'fluxnorm': norm}
cube = q3di.load_cube()

Let's plot the spectrum to see how it looks. The arguments are column and row in unity-offset units. 

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=[10,4])
spec_test = cube.specextract(1, 1, radius=0)

Here we are checking the variance, which is possibly too low. We adjust the range for this so that it's a clean continuum. RMS of the data should be on the same order of magnitude of the RMS of the variance itself.

In [ ]:
rms_data = np.sqrt(np.mean((cube.dat[2300:2900]-(np.median(cube.dat[2300:2900])))**2))
rms_variance = np.sqrt(np.mean((cube.var[2300:2900])**2))
fig = plt.figure(figsize=[10,4])
plt.plot(cube.dat[2300:2900])
plt.plot(cube.var[2300:2900])
print('RMS Data: %.5f RMS Variance Array: %.5f' % (rms_data, rms_variance))

Since the variance is much too low, we will boost it.

In [ ]:
cube.var *= 1e3
rms_data = np.sqrt(np.mean((cube.dat[2300:2900]-(np.median(cube.dat[2300:2900])))**2))
rms_variance = np.sqrt(np.mean((cube.var[2300:2900])**2))
fig = plt.figure(figsize=[10,4])
plt.plot(cube.dat[2300:2900])
plt.plot(cube.var[2300:2900])
print('RMS Data: %.5f RMS Variance Array: %.5f' % (rms_data, rms_variance))

Now re-write the variance-adjusted file, and have `q3di` point to the new file.

In [ ]:
newinfile = indir+'pds456-varadj.fits'
cube.writefits(newinfile)
q3di.infile = newinfile

Load and check the new file. Note that the input flux is different, because we saved it in the working units of erg/s/cm2/micron rather than the original input units of Jy.

In [ ]:
q3di.argsreadcube = {'wmapext': None,
                     'zerodq': True,
                     'fluxunit_in': 'erg/s/cm2/micron'}
cube = q3di.load_cube()
fig = plt.figure(figsize=[10,4])
spec_test = cube.specextract(1, 1, radius=0)

Name and systemic redshift of the galaxy. `zsys_gas` is an input for calculating velocity maps in `q3dpro` and for initializing the arrays of initial guesses below. In this case, the spectrum has already been shifted to the rest frame.

In [ ]:
q3di.name = 'PDS 456'
q3di.zsys_gas = 0.185

Wavelength range over which to fit data. The user can also specify sets of regions to ignore in the fit.

In [ ]:
q3di.fitrange = np.array([4.91 , 27.88])  # micron
#q3di.cutrange = np.array([,])

### 1.3. Setting up the fitting parameters <a class="anchor" id="chapter1_3"></a>

#### 1.3.1. Emission-line parameters <a class="anchor" id="chapter1_3_1"></a>

What lines do you want to fit? You can choose from the linelists [here](https://github.com/Q3D/q3dfit/tree/main/data/linelists), or in `q3dfit/data/linelists/`.

In [ ]:
lines = ['[ArII]6.99', '[NeII]12.81', 'H2_00_S2', '[NeIII]15.56', 'H2_00_S1', '[SIII]18.71']

This block sets up initial conditions for the emission-line fit to each spaxel. This initialization method adds a number of new attributes to the object. Emission lines are set to a common redshift and velocity dispersion, set to `q3di.zsys_gas` and 50 km/s by default. However, different sets of emission lines can have different velocities and linewidths by specifying different lines to which to tie particular emission lines. Different initial conditions can also be set on a spaxel-by-spaxel and/or line-by-line basis. The default number of velocity components is 1.

In [ ]:
q3di.init_linefit(lines, maxncomp=2)
q3di.__dict__.keys()

Here we set some initial guesses for linewidth and tie certain groups of lines together.

In [ ]:
for i in lines:
    q3di.siginit_gas[i][:,:,0] = 100.

q3di.linetie['[NeIII]15.56'] = '[NeII]12.81'
q3di.linetie['[SIII]18.71'] = '[NeII]12.81'
q3di.linetie['H2_00_S2'] = 'H2_00_S1'

`siglim_gas` sets lower and upper bounds for the Gaussian width (sigma) of the emission line. These limits can be set globablly, for all spaxels and components, by defining a 2-element array. The limits can also be set for individual spaxels (but all components) by defining an (Ncol x Nrow x 2) array.

In [ ]:
q3di.siglim_gas = np.array([1., 1500.])

# Spaxel-by-spaxel limit
# siglim_gas = np.ndarray((dx, dy, 2))
# siglim_gas[:,:,] = array([5.,1000.])
# siglim_gas[13, 10, :] = array([5.,500.])

The routine `checkcomp` automatically discards components that it deems insignificant after each fit. It does so with both a significance cut on flux, and if the linewidth is too large. If components are removed, the fit is re-run. The `sigcut` parameter determines the level of the significance cut. The `perror_useresid` option allows to substitute the formal line flux error with one estimated from the residual of the continnuum fit. This aids in more accurate component rejection in this case, because undersampling wiggles and fringeing are still present in the data and raise the actual error above that estimated by the pipeline (which is in any case too small). The `subone` option tells `checkcomp` to remove only one component at a time (necessary, e.g., if a two-component fit yields two low-significance components that both get rejected, but a one-component fit does not).

In [ ]:
q3di.checkcomp = True
q3di.argscheckcomp['sigcut'] = 2.
q3di.argscheckcomp['subone'] = True
q3di.perror_useresid = True

#### Spectral resolution convolution

If no convolution is desired: do not set `spect_convol` (or set it to `{}`, or `None`).

If convolution is desired: `spect_convol` is a dictionary with two optional tags.
- `ws_instrum`: This specifies the desired convolution method. The syntax is: `{INSTRUMENT:[GRATING]}`. The values for `INSTRUMENT` and `GRATING` for pre-defined dispersion files should mirror the filename syntax in `q3dfit/data/dispersion_files/`. E.g., for file `jwst_miri_ch1a_disp.fits`, `INSTRUMENT=jwst_miri` and `GRATING=ch1a`. (Case is irrelevant. For convolution with a constant value of spectral resolution [R], Δλ FWHM in [$\mu$m], or velocity in [km/s], set `INSTRUMENT = flat` and `GRATING = ` a string containing `R`, `dlam`, or `dvel` and the corresponding numerical quantity. More thana one instrument and/or grating can be set.
- `dispdir`: Directory in which to find the dispersion files. If not set, the default `q3dfit` directory is searched.

Examples: 
1. flat R=500: `spect_instrum = {'flat':['R500']}`
2. flat velocity FWHM = 30km/s: `spect_instrum = {'flat':['dvel30']}`
3. flat Δλ FWHM = 4 Å: `spect_instrum = {'flat':['dlam0.0004']}`
4. JWST NIRSPEC / G140M: `spect_instrum = {'JWST_NIRSPEC':['G140M']}`
5. Spitzer IRS SH+LH: `spect_instrum = {'Spitzer_IRS':['ch1_sh','ch1_lh']}`

Note in the final example that two gratings are set.

In [ ]:
q3di.spect_convol['ws_instrum'] = {'JWST_MIRI':['ch1a'], 'JWST_MIRI':['ch1b'], 'JWST_MIRI':['ch1c'],
                                   'JWST_MIRI':['ch2a'], 'JWST_MIRI':['ch2b'], 'JWST_MIRI':['ch2c'],
                                   'JWST_MIRI':['ch3a'], 'JWST_MIRI':['ch3b'], 'JWST_MIRI':['ch3c'],
                                   'JWST_MIRI':['ch4a'], 'JWST_MIRI':['ch4b'], 'JWST_MIRI':['ch4c']}

##### Creating convolution files (optional)

To create a dispersion file, use one of the following methods. The second two involve specific subclasses of the dispersion class used for the instrument/grating file or constant dispersion formats 

1. Create a `dispersion` object and use the `dispersion.write()` method. For example:

```
dispEx1 = dispersion()
dispEx1.write('/dispdir/disp.fits', wave=np.linspace(5.,10.,50), type='R', disp=np.full(50, 500.))
```

2. Create a `InstGratDispersion` object to attach instrument and grating information to the object and define the output filename in the `q3dfit` format. Use the `InstGratDispersion.writeInstGrat()` method.

```
dispEx2 = InstGratDispersion(`Keck_ESI`,`echellette`, dispdir=`/dispdir/`)
dispEx2.writeInstGrat(wave=np.linspace(5.,10.,50), type='dvel', disp=np.full(50, 30.))
```

3. Create a `FlatDispersion` object and use the `FlatDispersion.writeFlat()` method. This requires only a single value for the dispersion quantity and also defines the filename automatically.
```
dispEx3 = FlatDispersion(0.0004,`dlam`,wave=np.linspace(5.,10.,50))
dispEx3.writeFlat(dispdir=`/dispdir/`)
```

#### Options to `lmfit` and `scipy.optimize.least_squares`
`q3dfit` uses the `fit` method of the [`Model` class](https://lmfit.github.io/lmfit-py/model.html#lmfit.model.Model) of `lmfit` to call [`scipy.optimize.least_squares`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html). Both the method and function have options which can be changed in the `q3dfit` call. To do so, add key/value pairs to the `argslinefit` dictionary, which in turn is a keyword of the `q3di` dictionary.

The options to the `fit` method in `lmfit` that can currently be changed are the following:
- `max_nfev`: maximum number of function evaluations before the fit aborts
- `iter_cb`: if this is set to "per_iteration", the value of every model parameter at each function evaluation is printed to `stdout`

Most parameters of `least_squares` can be changed in this way, unless they are specifically set by `lmfit`. Examples which have been tested include:
- `x_scale`: jac
- `tr_solver`: lsmr
- `loss`: soft_l1
- `ftol`, `gtol`, `xtol`

In [ ]:
#q3di.argslinefit['method'] = 'leastsq'
#q3di.argslinefit['iter_cb'] = 'per_iteration'
# As an example, to change the criteria for fit convergence from the defaults of 1.e-8 to 1.e-10:
q3di.argslinefit['ftol'] = 1.e-10
q3di.argslinefit['gtol'] = 1.e-10
q3di.argslinefit['xtol'] = 1.e-10
#q3di.argslinefit['x_scale'] = 'jac'
#q3di.argslinefit['tr_solver'] = 'lsmr'
# .. and the "suitable step length for the forward- difference approximation of the Jacobian. Normally the actual step length will be sqrt(epsfcn)*x"
# this is only for scipy.optimize.leastsq
#q3di.argslinefit['epsfcn'] = 1.e-15

#### 1.3.2 Continuum parameters <a class="anchor" id="chapter1_3_2"></a>

We next initialize the continuum. As part of this, we give it the name of our continuum fitting function. (See Rupke et al. 2017 for more details on the methodology of `q3dfit` when separating a quasar from its host galaxy.)

In [ ]:
q3di.init_contfit('questfit')
q3di.__dict__.keys()

`q3dfit` first masks emission lines before fitting. This sets is the default mask value in km/s for each velocity component for the first fit. During the second fit, the mask value is set automatically using the best-fit linewidths determined from the first fit.

In [ ]:
q3di.maskwidths_def = 1500.

The continuum fitting parameters specified here are for the case of general MIR fitting. The mid-IR continuum fitting includes features that depend on redshift. These are specified as "stellar" redshift for compatibility with stellar template fitting, even though they refer in this case to the redshift of the mid-IR dust features. The input spectrum has in this case already been shifted to rest wavelengths.

In [ ]:
q3di.argscontfit['config_file'] = cfinfile
q3di.argscontfit['convert2Flambda'] = True
q3di.argscontfit['plot_decomp'] = True
q3di.argscontfit['outdir'] = outdir

Optional arguments to `lmfit`. These are the tolerances for determining fit convergence, described in further detail [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html).

In [ ]:
#argslmfit = dict()
#argslmfit['ftol'] = 1.e-10
#argslmfit['gtol'] = 1.e-10
#argslmfit['xtol'] = 1.e-10
#argslmfit['x_scale'] = 'jac'
#argslmfit['tr_solver'] = 'lsmr'
#q3di.argscontfit['argslmfit'] = argslmfit

If you want to run `q3dfit` in batch mode, run this cell, which saves q3di to an `npy` file. In your python command line, read in file and run `q3dfit` with
<pre><code>q3di = '/path/to/the/npy/file/q3di.npy'
from q3dfit.q3dfit import q3dfit
q3dfot(q3di,cols=cols,rows=rows)</code></pre>
N.B.: When running `q3dfit` using multiple cores (`ncores=N` in the call to `q3df`), the input dictionary has to be specified in this way; i.e., as a string describing the location of this .npy file.

In [ ]:
q3di_npy = 'q3di.npy'
np.save(initdir+q3di_npy, q3di)

## 2. Run fit <a class="anchor" id="chapter2"></a>

Choose columns and rows to fit. Ranges are specified as two-element lists specifying the first and last spaxel. Because there is only one spaxel in this case, we don't actually have to specify the rows and columns, but we'll do it to illustrate the syntax.

In [ ]:
cols = 1
rows = 1

Run the fit. Choose `quiet=False` for verbose output. An output object for each spaxel, of class `q3dout`, is saved to a numpy binary file labeled with prefix `q3di['label']` and suffix `_col_row.npy`. See note above on multicore processing.

In [ ]:
from q3dfit.q3df import q3dfit
q3dfit(q3di,cols=cols,rows=rows, quiet=False)

## 3. Plot fit results <a class="anchor" id="chapter3"></a>

Load the output of a fit.

In [ ]:
from q3dfit.q3dout import load_q3dout
q3do = load_q3dout(q3di, cols, rows)

Set up the line plot parameters using a dictionary.

* `nx`: Number of subplots in the horizontal direction (default = 1)
* `ny`: Number of subplots in the vertical direction (default = 1)
* Required: choose one options for centerting the plot
    - `line`: a string list of line labels
    - `center_obs`: a float list of wavelengths of each subplot center, in the observed (plotted) frame
    - `center_rest`: a float list of wavelengths of each subplot center, in the rest frame, which are converted to obs. frame
* `size`: float list of widths in wavelength space of each subplot; if not specified (default = 300 $Å$)

In [ ]:
argsplotline = dict()
argsplotline['nx'] = 3
argsplotline['ny'] = 2
argsplotline['line'] = ['[ArII]6.99', '[NeII]12.81', 'H2_00_S2', 
                        '[NeIII]15.56', 'H2_00_S1', '[SIII]18.71']
argsplotline['size'] = [.2, .4, .2, .6, .4, .5]
argsplotline['figsize'] = [10,8]

Run the plot method. The output can be saved as a jpg by specifying `savefig=True`. A default filename is used, which can be overridden by specifying `outfile=file`. The output file will have the suffix `_lin` attached, so that the actual filename will be "file_lin.jpg".

In [ ]:
q3do.plot_line(q3di, plotargs=argsplotline)

The continuum plot can be changed by specifying several parameters. In this case, we have chosen to output a log/log plot of f$_\nu$ vs. wavelength.

In [ ]:
argscontplot = dict()
#argscontplot['xstyle'] = 'lin'
#argscontplot['ystyle'] = 'lin'
#argscontplot['fluxunit_out'] = 'flambda'
argscontplot['mode'] = 'dark'
argscontplot['figsize'] = [10,8]

Run two methods. The first computes the continuum values to plot, and the second does the plotting.

In [ ]:
q3do.sepcontpars(q3di)
q3do.plot_cont(q3di, plotargs=argscontplot)

This is a custom plotter so that we can see the details of the continuum fit components.

In [ ]:
from scipy.interpolate import interp1d

def ergs2Jy(y):
    '''converts array from ergs to Jy
    uses the normalization from when the cube is initialized'''
    c = 2.99792458e14
    final = y*norm*q3do.wave**2/c
    return final*1e23

newerr = interp1d(cube.wave, cube.var)(q3do.wave)
chi = 0
chiarr = []
# Calculating X^2
for i in range(len(q3do.cont_dat)):
    obs = q3do.cont_fit[i]
    exp = q3do.cont_dat[i]
    err = newerr[i]
    chi += (obs-exp)**2/err
    chiarr.append((obs-exp)**2/err)
chi /= (len(q3do.spec)-5)
    
model_num = 6 # based on the config file
bb1_T =int(q3do.ct_coeff['MIRparams']['blackbody1350T'].value)
bb2_T =int(q3do.ct_coeff['MIRparams']['blackbody575T'].value)
bb3_T =int(q3do.ct_coeff['MIRparams']['blackbody50T'].value)
bb1 = q3do.ct_coeff['comp_best_fit']['blackbody1350']*q3do.ct_coeff['comp_best_fit']['blackbody1350_ext'] 
bb2 = q3do.ct_coeff['comp_best_fit']['blackbody575']*q3do.ct_coeff['comp_best_fit']['blackbody575_ext'] 
bb3 = q3do.ct_coeff['comp_best_fit']['blackbody50']*q3do.ct_coeff['comp_best_fit']['blackbody50_ext'] 

zwave = q3do.wave.copy()/(1+q3di.zsys_gas)
plt.style.use("default")
fig, (ax1, ax2) = plt.subplots(2, 1, height_ratios=[4, 1], figsize=(10, 9))
ax1.plot(zwave, ergs2Jy(q3do.cont_dat), label='data', linewidth=3)
ax1.plot(zwave, ergs2Jy(q3do.cont_fit), label='fit sum', color='tab:olive')
ax1.plot(zwave, ergs2Jy(q3do.ct_coeff['comp_best_fit']['template_0']), label='Silicate')
ax1.plot(zwave, ergs2Jy(q3do.ct_coeff['comp_best_fit']['template_1']), label='PAH')
ax1.plot(zwave, ergs2Jy(bb1), label=f'BB {bb1_T}K')
ax1.plot(zwave, ergs2Jy(bb2), label=f'BB {bb2_T}K')
ax1.plot(zwave, ergs2Jy(bb3), label=f'BB {bb3_T}K')


ax1.text(10, .5, '$\chi^2_{red}$:' + f' {round(chi, 4)}\n Model {model_num}')
ax1.set_ylabel('Flux (Jy)')

ax1.legend()

# Second axis plotting
ax2.set_xlabel('Rest Wavelength ($\mu m$)')
ax2.set_ylabel('Data/Model')
ax2.axhline(y=1, linestyle='--', linewidth=1, color='gray')
ax2.set_ylim(.8, 1.2)
ax2.plot(zwave, q3do.cont_dat/q3do.cont_fit, linewidth=1)
ax2.axvspan(5.1/1.185, 5.9/1.185, color='gray', alpha=0.5)  
ax2.axvspan(11.9/1.185, 12.6/1.185, color='gray', alpha=0.5)
ax2.axvspan(14.9/1.185, 15.3/1.185, color='gray', alpha=0.15) 
ax2.axvline(x=7.18/1.185, color='lightgray', zorder=-1)
ax2.axvline(x=12.12/1.185, color='lightgray', zorder=-1)
ax2.axvline(x=14.02/1.185, color='lightgray', zorder=-1)
ax2.axvline(x=16.2/1.185, color='lightgray', zorder=-1)
ax2.axvline(x=18.55/1.185, color='lightgray', zorder=-1)
ax2.axvline(x=21.67/1.185, color='lightgray', zorder=-1)
ax2.axvline(x=25.37/1.185, color='lightgray', zorder=-1)

#plt.savefig(outdir+obj_name+f'_model{model_num}_plot.png')

Here's a flux density plot of just the blackbody components.

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(q3do.wave*1.185, bb1, label='767K')
plt.plot(q3do.wave*1.185, bb2, label='197K')
plt.plot(q3do.wave*1.185, bb3, label='76K')

plt.axvline(x=2898/bb1_T)
plt.axvline(x=2898/bb2_T, color='tab:orange')
plt.axvline(x=q3do.wave[np.argmax(bb2)]*1.185, color='tab:orange', alpha=.5)
plt.axvline(x=2898/bb3_T, color='tab:green')
plt.legend()
plt.xlabel('Observed wavelength (um)')
plt.ylabel('Flux (erg/s/cm^2/um)')

Saving continuum fit profiles into text file for PAH analysis.

In [ ]:
# in units of Jy
# wave, model, PAH model, silicate model, data, data_err
np.savetxt(indir + q3di.name + '_continuum_fit_profiles.txt', 
           [q3do.wave, ergs2Jy(q3do.cont_fit), 
            ergs2Jy(q3do.ct_coeff['comp_best_fit']['template_1']), 
            ergs2Jy(q3do.ct_coeff['comp_best_fit']['template_0']), 
            ergs2Jy(q3do.cont_dat), 
            ergs2Jy(q3do.spec_err)])

Display a table of the mid-IR best-fit parameters.

In [ ]:
q3do.ct_coeff['MIRparams']